In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas as gpd
import seaborn as sns

sns.set()

%matplotlib inline

In [ ]:
data = pd.read_csv('../dataset/geo.csv', index_col=0)
data

Descartamos los de score menor a 90

In [ ]:
data = data[data['match_score'] >= 90]
data

In [ ]:
import math
countries = data['country'].value_counts()
countries = countries.to_frame().reset_index()
countries = countries.rename(columns={'index': 'country', 'country': 'tweets'})
countries

In [ ]:
main_countries = countries[:7]
main_countries = main_countries.append({'country': 'OTHERS', 'tweets': countries.loc[7:, 'tweets'].sum()}, ignore_index=True)
main_countries

In [ ]:
plt.style.use('Solarize_Light2')
fig1, ax1 = plt.subplots(figsize=(12, 12))
fig1.suptitle('Tweets with location by country', fontsize=25)
ax1.pie(main_countries['tweets'], labels=main_countries['country'], autopct='%1.1f%%',
        startangle=90, textprops={'fontsize': 16})
plt.savefig('tweets-with-location-by-country.png')
plt.show()

In [ ]:
cities = data['city'].value_counts()
cities

In [ ]:
main_cities = cities.head(8)

plt.style.use('Solarize_Light2')
fig1, ax1 = plt.subplots(figsize=(20, 10))
fig1.suptitle('Main cities', fontsize=25)
ax1.bar(main_cities.index, main_cities)
ax1.set_xlabel('Cities', fontsize=20)
ax1.set_ylabel('# tweets', fontsize=20)
ax1.tick_params(axis='both', labelsize=16)
plt.savefig('main-cities.png')
plt.show()

In [ ]:
shapefile = '../map/ne_50m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf.drop(gdf.index[239])
merged = gdf.merge(countries, left_on='country_code', right_on='country' , how='left')
merged.fillna(0, inplace = True)

In [ ]:
vmin, vmax = countries['tweets'].min(), countries['tweets'].max()
norm = colors.LogNorm(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(1, figsize=(15, 8))
merged.plot(column='tweets', cmap='YlGnBu', linewidth= 0.4 , ax=ax, edgecolor='0.3', norm=norm)
#Le saco los ejes
ax.axis('off')

#Titulo
ax.set_title('Tweets by country', fontdict={'fontsize': '25', 'fontweight' : '3'})

#Barra indicadora
sm = plt.cm.ScalarMappable(cmap='YlGnBu', norm=norm)
cbar = fig.colorbar(sm)

cbar.set_label('# tweets',size = 20)
plt.savefig('tweets-map.png')